In [1]:
import torch
import torch.nn as nn
import os
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, utils, models
from torch.nn import functional as F
from torch.autograd import Variable
from tensorboardX import SummaryWriter
from tqdm import trange, tqdm
from PIL import Image
import torch.utils.model_zoo as model_zoo

In [2]:
root_dir = 'tiny-imagenet-200/train/'
writer = SummaryWriter()
batch_size = 128
np.random.seed(42)

In [3]:
classes = os.listdir(root_dir)
test_classes = np.random.choice(classes, size = int(0.1 * len(classes)), replace = False)
classes = [i for i in classes if i not in test_classes]
n = len(classes)
test_n = len(test_classes)

In [4]:
class TestDataset(Dataset):
    def __init__(self, classes):
        self.images = []
        self.support_set = []
        self.n = len(classes)
        self.cur_class = 0
        self.cur_img = 0
        self.size = 0
        i = 0
        for cl in classes:
            self.images.append([])
            sup = np.random.choice(os.listdir(root_dir + cl + '/images/'))
            img = Image.open(root_dir + cl + '/images/' + sup)
            img = img.resize((299, 299))
            img = np.array(img, dtype = 'float32') / 255
            if not len(img.shape) == 3:
                img = np.stack([img, img, img])
            else:
                img = img.transpose((2, 0, 1))
            self.support_set.append(torch.Tensor(img))
            for img_name in os.listdir(root_dir + cl + '/images/'):
                if not sup == img_name:
                    i += 1
                    img = Image.open(root_dir + cl + '/images/' + img_name)
                    img = img.resize((299, 299))
                    img = np.array(img, dtype = 'float32') / 255
                    if not len(img.shape) == 3:
                        img = np.stack([img, img, img])
                    else:
                        img = img.transpose((2, 0, 1))
                    self.images[-1].append(img)
                    self.size += 1
                    if i > 20:
                        i = 0
                        break
        self.support_set = torch.stack(self.support_set)
        
    def __len__(self):
        return self.size
        
    def __getitem__(self):
        img1 = self.images[self.cur_class][self.cur_img]
        self.cur_img += 1
        if self.cur_img >= len(self.images[self.cur_class]):
            self.cur_img = 0
            self.cur_class = (self.cur_class + 1) % self.n
        
        sample = {'img1' : torch.from_numpy(img1.astype('float32')), 'res' : self.cur_class}
        return sample

In [5]:
model = models.Inception3().cuda()
model.eval()
model.load_state_dict(torch.load("/home/david/.torch/models/inception_v3_google-1a9a5a14.pth"))

In [7]:
images = []
test = []
for cl in tqdm(classes):
    images.append([])
    test.append([])
    for img_name in os.listdir(root_dir + cl + '/images/'):
        img = Image.open(root_dir + cl + '/images/' + img_name)
        img = img.resize((299, 299))
        img = np.array(img, dtype = 'float32') / 255
        if not len(img.shape) == 3:
            img = np.stack([img, img, img])
        else:
            img = img.transpose((2, 0, 1))
        img = model.forward(Variable(torch.Tensor(img)).unsqueeze(0).cuda())
        img = img.data.cpu().numpy()
        if np.random.uniform() < 0.1:
            test[-1].append(img)
        else:
            images[-1].append(img)

  0%|          | 0/180 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [21]:
np.save('images', images)
np.save('test', test)

In [6]:
images = np.load('images.npy')
test = np.load('test.npy')

In [7]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(3, 1))
    
    def forward(self, inp):
        return self.fc(inp)

In [8]:
model_fc = SiameseNetwork()
model_fc.cuda()
coef = 0.0

In [9]:
class ImageDataset(Dataset):
    def __init__(self, images, n_classes):
        self.images = images
        self.n = n_classes
        self.cur_class = 0
        self.cur_img = 0
        self.size = 0
        for i in images:
            self.size += len(i)
        
    def __len__(self):
        return self.size
        
    def __getitem__(self, idx):
        class1 = self.cur_class
        if np.random.uniform() < 0.5:
            class2 = np.random.randint(0, self.n)
            while class2 == class1:
                class2 = np.random.randint(0, self.n)
            res = 0
        else:
            class2 = class1
            res = 1
        
        img1 = self.images[class1][self.cur_img]
        img2 = self.images[class2][np.random.randint(0, high = len(self.images[class2]))]
        self.cur_img += 1
        if self.cur_img >= len(self.images[self.cur_class]):
            self.cur_img = 0
            self.cur_class = (self.cur_class + 1) % self.n
        
        sample = {'img1' : torch.from_numpy(img1), 'img2' : torch.from_numpy(img2), 'res' : res}
        return sample

In [10]:
dataset = ImageDataset(images, n)
dataloader = DataLoader(dataset, batch_size = batch_size)
test_dataset = ImageDataset(test, n)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

In [11]:
sgd = torch.optim.SGD(model_fc.parameters(), lr = 0.01, momentum = 0.9, weight_decay = coef)
criterion = nn.BCEWithLogitsLoss()
dist = nn.CosineSimilarity()
for i in trange(200):
    model_fc.train()
    for j, batch in enumerate(dataloader):
        target = Variable(batch['res']).float().cuda()
        img1 = batch['img1'].squeeze()
        img2 = batch['img2'].squeeze()
        cosine = dist(img1, img2)
        l1 = torch.sum(torch.abs(img1 - img2), dim = 1)
        l2 = torch.sum((img1 - img2) ** 2, dim = 1)
        inp = torch.stack([cosine, l1, l2], dim = 1)
        y_pred = model_fc.forward(Variable(inp).cuda())
        y_pred = y_pred.view(-1)
        loss = criterion(y_pred, target)
        mav = torch.mean(torch.abs(y_pred))
        writer.add_scalar('siamese cross entropy', loss, i * len(dataloader) + j)
        writer.add_scalar('siamese absolute value', mav, i * len(dataloader) + j)
        sgd.zero_grad()
        loss.backward()
        sgd.step()
    model_fc.eval()
    for j, batch in enumerate(test_dataloader):
        target = Variable(batch['res']).float().cuda()
        img1 = batch['img1'].squeeze()
        img2 = batch['img2'].squeeze()
        cosine = dist(img1, img2)
        l1 = torch.sum(torch.abs(img1 - img2), dim = 1)
        l2 = torch.sum((img1 - img2) ** 2, dim = 1)
        inp = torch.stack([cosine, l1, l2], dim = 1)
        y_pred = model_fc.forward(Variable(inp).cuda())
        y_pred = y_pred.view(-1)
        loss = criterion(y_pred, target)
        writer.add_scalar('test siamese cross entropy', loss, i * len(test_dataloader) + j)

100%|██████████| 200/200 [14:30<00:00,  4.35s/it]


In [12]:
dataset = TestDataset(test_classes)

In [19]:
correct_a = 0
correct_b = 0
dist = nn.CosineSimilarity()
sup = model.forward(Variable(dataset.support_set).cuda())
for i in trange(len(dataset)):
    sample = dataset.__getitem__()
    target = sample['res']
    y_pred = model.forward(Variable(sample['img1']).unsqueeze(0).cuda())
    cosine = dist(y_pred, sup)
    l1 = torch.sum(torch.abs(sup - y_pred), dim = 1)
    l2 = torch.sum((sup - y_pred) ** 2, dim = 1)
    inp = torch.stack([cosine, l1, l2], dim = 1)
    a = model_fc.forward(inp.cuda())
    b = dist(y_pred, sup)
    c = torch.abs(sup - y_pred).data.cpu().numpy()
    c = np.max(c, axis = 1)
    pred_a = np.argmax(a.data.cpu().numpy())
    pred_b = np.argmax(b.data.cpu().numpy())
    pred_c = np.argmin(c)
    if target == pred_a:
        correct_a += 1
    if target == pred_b:
        correct_b += 1
    if target == pred_c:
        correct_c += 1
print(correct_a / len(dataset), correct_b / len(dataset), correct_b / len(dataset))

  4%|▎         | 15/420 [00:00<00:05, 69.33it/s]

(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)


  7%|▋         | 31/420 [00:00<00:05, 72.18it/s]

(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)


  9%|▉         | 39/420 [00:00<00:05, 72.49it/s]

...

 31%|███▏      | 132/420 [00:01<00:04, 71.53it/s]

(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)


 33%|███▎      | 140/420 [00:01<00:03, 71.59it/s]

(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)
(20,)


KeyboardInterrupt: 

In [16]:
for i in model_fc.parameters():
    print(i)

Parameter containing:
 207.5373   71.4570  -43.9818
[torch.cuda.FloatTensor of size 1x3 (GPU 0)]

Parameter containing:
-69.8412
[torch.cuda.FloatTensor of size 1 (GPU 0)]

